In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [3]:
movies_df = pd.read_csv('top10K-TMDB-movies.csv')
movies_df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


In [5]:
movies_df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [6]:
movies_df.isnull().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

### Feature Selection

In [7]:
movies_df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [8]:
movies_df = movies_df[["id", "genre","overview","title"]]

In [9]:
movies_df.head()

,id,genre,overview,title
0,278,"Drama,Crime",Framed in the 1940s for the double murder of h...,The Shawshank Redemption
1,19404,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",Dilwale Dulhania Le Jayenge
2,238,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",The Godfather
3,424,"Drama,History,War",The true story of how businessman Oskar Schind...,Schindler's List
4,240,"Drama,Crime",In the continuing saga of the Corleone crime f...,The Godfather: Part II


In [10]:
movies_df["tags"] = movies_df["overview"] + movies_df["genre"]

In [11]:
movies_df.head()

,id,genre,overview,title,tags
0,278,"Drama,Crime",Framed in the 1940s for the double murder of h...,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,"Comedy,Drama,Romance","Raj is a rich, carefree, happy-go-lucky second...",Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,"Drama,Crime","Spanning the years 1945 to 1955, a chronicle o...",The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,"Drama,History,War",The true story of how businessman Oskar Schind...,Schindler's List,The true story of how businessman Oskar Schind...
4,240,"Drama,Crime",In the continuing saga of the Corleone crime f...,The Godfather: Part II,In the continuing saga of the Corleone crime f...


In [12]:
new_df = movies_df.drop(columns=["overview","genre"], axis=1)
new_df.head()

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000, stop_words='english')
vectors = cv.fit_transform(new_df["tags"].values.astype("U")).toarray()
vectors.shape

(10000, 10000)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity.shape

(10000, 10000)

In [18]:
new_df[new_df["title"] == "Spider-Man: Into the Spider-Verse"].index[0]

np.int64(35)

In [29]:
dist = sorted(list(enumerate(similarity[2])),reverse=True, key=lambda vectors: vectors[1])
for i in dist[0:6]:
    print(new_df.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City
Gotti


In [30]:
def recommend(movie):
    index = new_df[new_df["title"] == movie].index[0]
    dist = sorted(list(enumerate(similarity[index])),reverse=True, key=lambda vectors: vectors[1])
    for i in dist[0:6]:
        print(new_df.iloc[i[0]].title) 

In [31]:
recommend("Spider-Man: Into the Spider-Verse")

Spider-Man: Into the Spider-Verse
Spider-Man
Spider-Man: No Way Home
Spider-Man: Homecoming
Batman: Gotham Knight
Deathstroke: Knights & Dragons - The Movie


In [33]:
import pickle
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(new_df,open("similarity.pkl", 'wb'))
pickle.load(open('movies.pkl', 'rb'))

,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...
...,...,...,...
9995,10196,The Last Airbender,"The story follows the adventures of Aang, a yo..."
9996,331446,Sharknado 3: Oh Hell No!,The sharks take bite out of the East Coast whe...
9997,13995,Captain America,"During World War II, a brave, patriotic Americ..."
9998,2312,In the Name of the King: A Dungeon Siege Tale,A man named Farmer sets out to rescue his kidn...
